In [1]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
import jax.numpy as jnp
import jax_smi
jax_smi.initialise_tracking()
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [2]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained(("models/llama-3-70b-1.gguf", "models/llama-3-70b-2.gguf"),
                                         device_map="auto:mp=4")

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("nvidia/Llama3-ChatQA-1.5-70B")
prompt = tokenizer.apply_chat_template([
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
], tokenize=False)

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
from penzai.toolshed import jit_wrapper
from micrlhf.scan import sequential_to_scan
from micrlhf.flash import flashify
tokens = pz.nx.wrap([(tokenizer.encode(prompt) * 8)[:128]], "batch", "seq")
inputs = llama.inputs.from_basic_segments(tokens)
# fast_llama = sequential_to_scan(flashify(llama))
fast_llama = sequential_to_scan(llama)
llama_jitted = jit_wrapper.Jitted(fast_llama)

In [13]:
llama_jitted(inputs);

XlaRuntimeError: RESOURCE_EXHAUSTED: XLA:TPU compile permanent error. Ran out of memory in memory space hbm. Used 58.98G of 30.75G hbm. Exceeded hbm capacity by 28.23G.

Total hbm usage >= 60.23G:
    reserved          1.25G 
    program          58.98G 
    arguments            0B 

Output size 0B; shares 0B with arguments.

Program hbm requirement 58.98G:
    global            16.5K
    HLO temp         58.98G (100.0% utilization: Unpadded (58.98G) Padded (58.98G), 0.0% fragmentation (4.42M))

  Largest program allocations in hbm:

  1. Size: 17.50G
     Shape: s8[80,8192,896,32]{1,3,2,0:T(8,128)(4,1)}
     Unpadded size: 17.50G
     XLA label: copy.134 = copy(param.20)
     Allocation type: HLO temp
     ==========================

  2. Size: 17.50G
     Shape: s8[80,28672,256,32]{1,3,2,0:T(8,128)(4,1)}
     Unpadded size: 17.50G
     XLA label: copy.133 = copy(param.18)
     Allocation type: HLO temp
     ==========================

  3. Size: 17.50G
     Shape: s8[80,28672,256,32]

In [ ]:
from micrlhf.sampling import sample
sample(llama, tokenizer, prompt, batch_size=1, max_seq_len=128, do_sample=True,
       fold=True,
       )